In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
WITH user_data AS (
    SELECT
        u.user_id,
        u.lt_day,
        CASE WHEN u.lt_day <= 365 THEN 1 ELSE 0 END AS is_new,  -- Флаг нового клиента
        u.age,
        u.gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        a.title AS age_segment,
        t.title AS traffic_segment,
        lt.title AS lifetime_segment,
        u.nps_score,
        CASE 
            WHEN u.nps_score >= 9 THEN 'Промоутер'
            WHEN u.nps_score <= 6 THEN 'Критик'
            ELSE 'Нейтрал'
        END AS nps_group  -- Добавлено поле NPS-группы
    FROM user u
    LEFT JOIN location l ON u.location_id = l.location_id
    LEFT JOIN age_segment a ON u.age_gr_id = a.age_gr_id
    LEFT JOIN traffic_segment t ON u.tr_gr_id = t.tr_gr_id
    LEFT JOIN lifetime_segment lt ON u.lt_gr_id = lt.lt_gr_id
)

SELECT * FROM user_data;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,Промоутер
1,A001WF,2344,0,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,Промоутер
2,A003Q7,467,0,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,Промоутер


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

In [6]:
from IPython.display import FileLink
FileLink('telecomm_csi_tableau.csv')

/home/jovyan/work/telecomm_csi_tableau.csv